In [27]:
import pandas as pd
import numpy as np
import geopandas as gpd
import math

from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.io import output_notebook 
from bokeh.palettes import Category20


pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

## Carga de datos

In [2]:
hogares = pd.read_csv('../data/Encuesta de Movilidad 2019/BD EODH2019 FINAL v14022020/Archivos CSV/HogaresEODH2019.csv', sep=';')
personas = pd.read_csv('../data/Encuesta de Movilidad 2019/BD EODH2019 FINAL v14022020/Archivos CSV/PersonasEODH2019.csv',sep=';')
etapas = pd.read_csv('../data/Encuesta de Movilidad 2019/BD EODH2019 FINAL v14022020/Archivos CSV/EtapasEODH2019.csv',sep=';')
viajes = pd.read_csv('../data/Encuesta de Movilidad 2019/BD EODH2019 FINAL v14022020/Archivos CSV/ViajesEODH2019.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
hogares_persona = pd.merge(hogares, personas, left_on='Id_Hogar', right_on='id_hogar')

## procesamiento datos hospitales

In [4]:
ideca = gpd.read_file('../data/salud/salud_ideca.geojson')
hospitales = ideca[ideca.NGeClasifi=='SALUD']
hospitales = hospitales[(hospitales.NGeNombre.str.contains('Hospital'))
                         |(hospitales.NGeNombre.str.contains('Clínica'))
                         |(hospitales.NGeNombre.str.contains('Clínica'))
                         |(hospitales.NGeNombre.str.contains('Fundación'))
                         |(hospitales.NGeNombre.str.contains('Fundacion'))].reset_index(drop=True)
hospitales['ZAT'] = 0
zats = gpd.read_file('../data/ZONAS/ZAT.shp')

for hindex, hrow in hospitales.iterrows():
    for zindex, zrow in zats.iterrows():
        if(hrow.geometry.within(zrow.geometry)):
            hospitales.loc[hindex,'ZAT'] = zrow.ZAT
            
hospitales.ZAT = hospitales.ZAT.astype(int)            
hospitales['longitud'] = hospitales.geometry.x
hospitales['latitud'] = hospitales.geometry.y

hospitales['departamento'] = 'Bogota, D.C.'
hospitales['municipio'] = 'Bogota, D.C.'
hospitales['cod_prestador'] = '1100100032'
hospitales['prestador'] = hospitales.NGeNombre
hospitales['cod_habilitacion'] = '1100100032'
hospitales['nombre_sede'] = hospitales.NGeNombre
hospitales['cod_sede'] = hospitales.NGeIdentif
hospitales['gerente'] = ''
hospitales['zona'] = 'Urbana'

hospitales = hospitales[['departamento', 'municipio', 'cod_prestador', 'prestador', 'cod_habilitacion', 
            'cod_sede', 'nombre_sede', 'gerente', 'zona', 'ZAT', 'latitud', 'longitud']]

hospitales.to_csv('../data/data_outputs/hospitales.csv')

## Procesamiento datos encuesta

In [5]:
personas_salud = personas[personas['p7_id_actividad_economica'].isin([17])
                         ][['id_hogar', 'id_persona', 'p6_id_ocupacion',
                            'p7_id_actividad_economica','p4_edad','Sexo', 'f_exp']]

In [6]:
viajes_persona = pd.merge(personas_salud, viajes[['id_hogar', 'id_persona', 'id_viaje','fecha', 'zat_origen', 
                                                  'lugar_origen', 'p17_Id_motivo_viaje', 'hora_inicio_viaje', 
                                                  'p28_lugar_destino', 'zat_destino', 'p31_hora_llegada', 
                                                  'modo_principal']], 
                          on=['id_hogar', 'id_persona'])

viajes_persona = viajes_persona[~viajes_persona.zat_origen.isna()]
viajes_persona = viajes_persona[~viajes_persona.zat_destino.isna()]

viajes_persona.zat_destino = viajes_persona.zat_destino.astype(int)
viajes_persona.zat_origen = viajes_persona.zat_origen.astype(int)

## Viajes usuales vs viajes esenciales

In [137]:
personas_esenciales = personas[personas['p7_id_actividad_economica'].isin([17,1,5,5,9,10,15,17 ])
                         ][['id_hogar', 'id_persona', 'p6_id_ocupacion',
                            'p7_id_actividad_economica','p4_edad','Sexo', 'f_exp']]
viajes_esenciales = pd.merge(personas_esenciales, viajes[['id_hogar', 'id_persona', 'id_viaje','fecha', 'zat_origen', 
                                                  'lugar_origen', 'p17_Id_motivo_viaje', 'hora_inicio_viaje', 
                                                  'p28_lugar_destino', 'zat_destino', 'p31_hora_llegada', 
                                                  'modo_principal']], 
                          on=['id_hogar', 'id_persona'])
viajes_esenciales = viajes_esenciales[viajes_esenciales['p17_Id_motivo_viaje'].isin([1,2])]

viajes_esenciales[viajes_esenciales.modo_principal=='TransMilenio'].f_exp.sum()/viajes[viajes.modo_principal=='TransMilenio'].f_exp.sum()

0.052722136526135534

## Viajes día normal

In [138]:
viajes.f_exp.sum()

18995614.913237266

## Viajes día de cuarentena

In [140]:
viajes_esenciales.f_exp.sum()

576266.5273336731

## Viajes Transmilenio día normal

In [141]:
viajes[viajes.modo_principal=='TransMilenio'].f_exp.sum()

2489737.539395058

## Viajes Transmilenio cuarentena

In [142]:
viajes_esenciales[viajes_esenciales.modo_principal=='TransMilenio'].f_exp.sum()

131264.282466231

In [8]:
dicc_motivo_viaje = {
    1: 'Trabajo/asuntos de trabajo',
    2: 'Trabajo/asuntos de trabajo',
    3: 'Estudiar',
    4: 'Recibir atención en salud',
    5: 'Visitar a alguien',
    6: 'Volver a casa',
    7: 'Dejar a alguien/algo',
    8: 'Dejar a alguien/algo',
    9: 'Ocio/recreación/comida/deporte',
    10: 'Ocio/recreación/comida/deporte',
    11: 'Tramites', 
    12: 'Ocio/recreación/comida/deporte',
    13: 'Buscar empleo',
    14: 'Actividades con fines religiosos',
    15: 'Cuidado de personas',
    16: 'Ocio/recreación/comida/deporte',
    77: 'Otro'
    
}

In [9]:
viajes['motivo'] = viajes.p17_Id_motivo_viaje.map(dicc_motivo_viaje)

In [10]:
viajes_modo = viajes[viajes.modo_principal.isin(['SITP Provisional', 'Transporte informal', 'SITP Zonal',
       'Bicicleta', 'Auto', 'Moto','TransMilenio','Transporte publico individual', 'Alimentador'])]

## Cantidad de viajes por modo de transporte por motivo de viaje

In [130]:
datos = pd.pivot_table(viajes_modo, values='f_exp', 
                       index='motivo', columns='modo_principal', aggfunc=np.sum).reset_index()

output_file("bar_stacked.html")
output_notebook()
motivo = datos.motivo
modo = datos.columns[1:].to_list()
colors = Category20[len(modo)]

data = {}

for col in datos.columns:
    data[col] = datos[col].to_list()

p = figure(x_range=motivo, plot_height=250, title="Cantidad de viajes por modo de transporte por motivo de viaje",
           toolbar_location=None, tools="hover", tooltips="$name @motivo: @$name")

p.vbar_stack(modo, x=datos.columns[0], width=0.9, color=colors, source=data,
             legend_label=modo)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

p.title.text_font_size = '15pt'
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.legend.label_text_font_size = "15pt"

p.xaxis.major_label_orientation = math.pi/2
p.plot_height=900
p.plot_width=1500
show(p)

Loading BokehJS ...

In [172]:
datos

modo_principal,motivo,A pie,Alimentador,Auto,Bicicleta,Bicitaxi,Intermunicipal,Moto,Otro,Patineta,SITP Provisional,SITP Zonal,TransMilenio,Transporte Escolar,Transporte informal,Transporte publico individual
0,Actividades con fines religiosos,0.000000,NaN,0.00000,0.000000,NaN,0.00000,0.000000,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN
1,Buscar empleo,0.000000,0.000000,NaN,0.000000,NaN,0.00000,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,NaN
2,Cuidado de personas,0.000000,NaN,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,0.000000
3,Dejar a alguien/algo,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Estudiar,0.000000,NaN,0.00000,0.000000,0.00000,NaN,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,NaN,0.000000,0.000000
5,Ocio/recreación/comida/deporte,0.000000,0.000000,0.00000,0.000000,NaN,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000
6,Otro,0.000000,NaN,0.00000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000
7,Recibir atención en salud,0.000000,0.000000,0.00000,0.000000,NaN,0.00000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000
8,Trabajo/asuntos de trabajo,73225.025366,5013.822837,76594.63983,45784.357485,270.25584,20045.01613,63583.942567,21387.202742,357.110546,41313.296889,59247.618735,131264.282466,291.144598,18067.412882,19821.398422
9,Tramites,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,NaN,0.000000,0.000000


# viajes por modo en cuarentena

In [183]:
viajes_esenciales['motivo'] = viajes_esenciales.p17_Id_motivo_viaje.map(dicc_motivo_viaje)
viajes_2 = pd.merge(viajes, personas_esenciales, on=['id_hogar', 'id_persona'])
viajes_2['f_exp'] = np.where(viajes_2.p17_Id_motivo_viaje.isin([1,2]), viajes_2.f_exp_y, 0)

datos = pd.pivot_table(viajes_2, values='f_exp', 
                       index='motivo', columns='modo_principal', aggfunc=np.sum).reset_index()

datos.loc[datos['motivo']=='Volver a casa']

output_file("bar_stacked.html")
output_notebook()
motivo = datos.motivo
modo = datos.columns[1:].to_list()
colors = Category20[len(modo)]

data = {}

for col in datos.columns:
    data[col] = datos[col].to_list()

p = figure(x_range=motivo, plot_height=250, title="Cantidad de viajes por modo de transporte por motivo de viaje",
           toolbar_location=None, tools="hover", tooltips="$name @motivo: @$name")

p.vbar_stack(modo, x=datos.columns[0], width=0.9, color=colors, source=data,
             legend_label=modo)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

p.title.text_font_size = '15pt'
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.legend.label_text_font_size = "15pt"

p.xaxis.major_label_orientation = math.pi/2
p.plot_height=900
p.plot_width=1500
show(p)

Loading BokehJS ...

## Cantidad de viajes por modo de transporte por motivo de viaje (solo esenciales)

In [102]:
viajes_esenciales['motivo'] = viajes_esenciales.p17_Id_motivo_viaje.map(dicc_motivo_viaje)
viajes_esenciales = viajes_esenciales[viajes_esenciales.p17_Id_motivo_viaje.isin([1,2])]
viajes_esenciales = viajes_esenciales[~viajes_esenciales.modo_principal.isin(['A pie', 'Intermunicipal'])]

In [131]:
datos = pd.pivot_table(viajes_esenciales, values='f_exp', 
                       index='modo_principal', columns='motivo', aggfunc=np.sum).reset_index()

from bokeh.plotting import figure
from bokeh.transform import cumsum
from bokeh.palettes import Category20c
from bokeh.models import LabelSet, ColumnDataSource

output_file("pie.html")
output_notebook()

datos['perc'] = 100*datos['Trabajo/asuntos de trabajo']/datos['Trabajo/asuntos de trabajo'].sum()
datos = datos[datos.perc>=1]
datos.perc = datos.perc.astype(int)
datos.perc = datos.perc.astype(str)
datos.perc = datos.perc + ' %'
datos["perc"] = datos["perc"].str.pad(45, side = "left")



data = datos
data['angle'] = data['Trabajo/asuntos de trabajo']/data['Trabajo/asuntos de trabajo'].sum() * 2*math.pi
data['color'] = Category20[len(data.modo_principal.to_list())]

p = figure(plot_height=350, title="Distribución modal de viajes esenciales", toolbar_location=None,
           tools="hover", tooltips="@modo_principal: @Trabajo/asuntos de trabajo", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='modo_principal', source=data)

source = ColumnDataSource(data)
labels = LabelSet(x=0, y=1, text='perc',
        angle=cumsum('angle', include_zero=True), source=source, render_mode='canvas')


p.add_layout(labels)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None

p.title.text_font_size = '15pt'
p.plot_height=900
p.plot_width=900

show(p)

Loading BokehJS ...

In [104]:
data.modo_principal.unique()

array(['Alimentador', 'Auto', 'Bicicleta', 'Bicitaxi', 'Moto', 'Otro',
       'Patineta', 'SITP Provisional', 'SITP Zonal', 'TransMilenio',
       'Transporte Escolar', 'Transporte informal',
       'Transporte publico individual'], dtype=object)

In [132]:
datos = pd.pivot_table(viajes_esenciales, values='f_exp', 
                       index='modo_principal', columns='motivo', aggfunc=np.sum).reset_index()

from bokeh.plotting import figure
from bokeh.transform import cumsum
from bokeh.palettes import Category20c

output_file("pie.html")
output_notebook()

datos['perc'] = 100*datos['Trabajo/asuntos de trabajo']/datos['Trabajo/asuntos de trabajo'].sum()
data = datos
data['tp'] = np.where(data.modo_principal.isin(['Alimentador','SITP Provisional',
       'SITP Zonal', 'TransMilenio']), 'Transporte masivo', 'Otro')




data = data.groupby('tp').sum().reset_index()

data = data[data.perc>=1]
data.perc = data.perc.astype(int)
data.perc = data.perc.astype(str)
data.perc = data.perc + ' %'
data["perc"] = data["perc"].str.pad(45, side = "left")


data['angle'] = data['Trabajo/asuntos de trabajo']/data['Trabajo/asuntos de trabajo'].sum() * 2*math.pi
data['color'] = ['#00cccc', '#f9ac26']

p = figure(plot_height=350, title="Distribución modal de viajes esenciales", toolbar_location=None,
           tools="hover", tooltips="@tp: @Trabajo/asuntos de trabajo", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='tp', source=data)

source = ColumnDataSource(data)
labels = LabelSet(x=0, y=1, text='perc',
        angle=cumsum('angle', include_zero=True), source=source, render_mode='canvas')


p.add_layout(labels)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None

p.title.text_font_size = '15pt'
p.plot_height=900
p.plot_width=900

show(p)

Loading BokehJS ...

In [143]:
data

motivo,tp,Trabajo/asuntos de trabajo,perc,angle,color
0,Otro,246157.464911,50 %,3.202203,#00cccc
1,Transporte masivo,236839.020927,49 %,3.080982,#f9ac26


## Viajes día normal

In [133]:
viajes.f_exp.sum()

18995614.913237266

## Viajes día de cuarentena

In [134]:
viajes_esenciales.f_exp.sum()

482996.48583790206